In [1]:
import re
from dataclasses import dataclass

import brightway2 as bw

bw.projects.set_current("EF calculation")


@dataclass
class CopyContext:
    copy_in: str
    copy_from: str
    string_replacement: (str, str)

    def __post_init__(self):
        self.matches = {}
        self.not_matched_copies = []
        self.copies = [
            x
            for x in bw.Database(self.copy_in)
            if re.search(self.string_replacement[0], x["name"], re.IGNORECASE)
        ]


contexts = (
    CopyContext(
        copy_in="agribalyse3",
        copy_from="ecoinvent_3.8_cutoff_from_sp",
        string_replacement=("Cut-off, S - Copied from Ecoinvent", "Cut-off, U"),
    ),
    CopyContext(
        copy_in="agribalyse3",
        copy_from="WFLDB",
        string_replacement=(r"\(WFLDB \d\.\d\)", "(WFLDB)"),
    ),
    CopyContext(
        copy_in="WFLDB",
        copy_from="ecoinvent_3.8_cutoff_from_sp",
        string_replacement=("Cut-off, S - Copied from ecoinvent", "Cut-off, U"),
    ),
)

In [2]:
from tqdm.notebook import tqdm

for context in contexts:
    for copy_act in tqdm(context.copies):
        try:
            original_act = next(
                act
                for act in bw.Database(context.copy_from)
                if (
                    act["name"].lower()
                    == re.sub(
                        *context.string_replacement, string=copy_act["name"]
                    ).lower()
                )
                and (act.get("location") == copy_act.get("location"))
            )
            context.matches[copy_act] = original_act

        except StopIteration:
            context.not_matched_copies.append(copy_act)

  0%|          | 0/1366 [00:00<?, ?it/s]

  0%|          | 0/372 [00:00<?, ?it/s]

  0%|          | 0/663 [00:00<?, ?it/s]

In [3]:
for context in contexts:
    print(
        f"{context.copy_in} <- {context.copy_from}: {len(context.matches) / (len(context.copies)):.0%}"
    )

agribalyse3 <- ecoinvent_3.8_cutoff_from_sp: 97%
agribalyse3 <- WFLDB: 52%
WFLDB <- ecoinvent_3.8_cutoff_from_sp: 97%


Asserting the GWP score of original and copy are similar

In [33]:
import pandas as pd

method = ("IPCC 2013", "climate change", "GWP 100a")

scores = []
for context in contexts:
    copy_lca = bw.LCA({list(context.matches.keys())[0]: 1}, method)
    copy_lca.lci()
    copy_lca.lcia()
    original_lca = bw.LCA({list(context.matches.values())[0]: 1}, method)
    original_lca.lci()
    original_lca.lcia()

    for copy_act, original_act in tqdm(context.matches.items()):
        copy_lca.redo_lcia({copy_act: 1})
        original_lca.redo_lcia({original_act: 1})
        relative_difference = (
            ((copy_lca.score - original_lca.score) / original_lca.score)
            if original_lca.score != 0
            else 0
        )

        scores.append(
            {
                "copy_in": context.copy_in,
                "copy_from": context.copy_from,
                "copy": copy_act.key,
                "original": original_act.key,
                "copy_score": copy_lca.score,
                "original_score": original_lca.score,
                "relative_difference": relative_difference,
            }
        )

df = pd.DataFrame(scores)
df.to_csv("matches_differences.csv", index=False)

  0%|          | 0/1319 [00:00<?, ?it/s]

  0%|          | 0/194 [00:00<?, ?it/s]

  0%|          | 0/646 [00:00<?, ?it/s]